In [ ]:
%matplotlib inline
%matplotlib widget
from datetime import datetime, timedelta 
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import asyncio
import os
from lsst.ts import salobj 
from scipy.fft import fft, fftfreq
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

from lsst_efd_client import EfdClient


In [ ]:
async def injectForce_axial_actuator(csc, actuators, force, sleep_time=5):
    """Bump the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    actuators : list of actuators.
    force : list of forces
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Do the positive direction first
    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0.] * num_axial_actuator
    
    for idx in range(len(actuators)):
        print(f"idx: {idx}.")
        index = actuators[idx]
        #print(f"idx: {idx}  index  {index}")        
        forces[index] = force[idx]
        print(f"Apply the force: {force} N. to actuator: {index}")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)


In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
"""    
DISABLED = 1
ENABLED = 2
FAULT = 3
OFFLINE = 4
STANDBY = 5

Standby  -->  Disable  -->  Enabled

await m2.cmd_start.set_start(timeout=30)
await m2.cmd_enable.set_start(timeout=200)

3) To Stop, do this:    Enabled  -->  Disable  -->  Standby

await m2.cmd_disable.set_start(timeout=30)
await m2.cmd_standby.set_start(timeout=30) """ 

In [ ]:
# get status
state = m2.evt_summaryState.get()
print(state)

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=30)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
def retrieve_efd_client(is_summit=True):
    """
    Retrieves a client to EFD.

    Parameters
    ----------
    is_summit : bool, optional
        This notebook is running on the summit or not. If not, the returned object will point
        to the test stand at Tucson.

    Returns
    -------
    EfdClient : The interface object between the Nublado and summit/Tucson EFD.
    """
    efd_name = "summit_efd" if is_summit else "tucson_teststand_efd"        
    return EfdClient(efd_name)


In [ ]:
efd_client = retrieve_efd_client(is_summit=True)


#fill a list with n actuators
n = 12

# for each group of actuator and each run remove comment related to the actuator group desired. Use only one group at time

actuators = [  0,  10,  20,   4,  14,  24,  31,  39,  47,  56,  62,  68]
#actuators = [  1,  11,  21,   5,  15,  25,  32,  40,  48,  57,  63,  69]
#actuators = [  2,  12,  22,   6,  16,  26,  33,  38,  46,  54,  63,  69]
#actuators = [  3,  13,  23,   7,  17,  27,  33,  41,  49,  58,  64,  70]
#actuators = [  4,  14,  24,   8,  18,  28,  34,  42,  50,  59,  64,  70]
#actuators = [  6,  16,  26,   9,  19,  29,  35,  43,  51,  60,  65,  71]
#actuators = [  6,  16,  26,  10,  20,  30,  36,  44,  52,  61,  66,   0]
#actuators = [  7,  17,  27,  11,  21,  31,  37,  45,  53,  55,  67,   0]


for idx in range(n):
    print(f"actuator: {actuators[idx]}")

In [ ]:
#bring all actuators to zero   
n_act = NUM_ACTUATOR - NUM_TANGENT_LINK
all_actuators = [0] * n_act
f_Delta = [0] * n_act
f_Delta = np.array(f_Delta, float)
forces = f_Delta

#apply force of the loop
for idx in range(n_act):
    force = forces[idx]
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(5)

In [ ]:
# retrieve from EFD for at least 120 sec of the measured forces on actuators to build a 
# force statistics

await asyncio.sleep(125)    


time_end = datetime.now()
time_end = Time(time_end, scale="utc")
time_start = time_end - timedelta(seconds = 120)
time_start = Time(time_start, scale="utc")

names = list()
for idx in range(n_act):
    names.append(f"measured{idx}")
    
data_act_before = await efd_client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=names,
        start=time_start,
        end=time_end)

# retrieve statistics
f_max = data_act_before.max()
f_min = data_act_before.min()
f_max_values = f_max.to_numpy()
f_min_values = f_min.to_numpy()
p_v = f_max_values - f_min_values
print(f"p_v : {p_v}")

In [ ]:
#Calculate the Delta forces to be applied, check for limits and applied them


data_measured_forces = m2.tel_axialForce.get()

data_measured_forces_values = np.array(data_measured_forces.measured, float)

# calculate force to be applied
f_Delta = 400 - (p_v[actuators] + data_measured_forces_values[actuators])

n_act_odd = int(n/2)

for idx in range(n_act_odd):
    f_Delta[idx*2+1] = -f_Delta[idx*2+1]
    print(f"value : {idx} {idx*2+1} {actuators[idx*2+1]} {f_Delta[idx*2+1]} ")

print(f"all actuators")
    
for idx in range(n):
    print(f"value : {idx} {actuators[idx]} {f_Delta[idx]} ")
        
total_force = np.sum(f_Delta)    
print(f"Total force : {total_force}")



#apply force of the loop
for idx in range(n):
    print(f"The actuator activated is  {actuators[idx]} N. to actuator: {f_Delta[idx]} {data_measured_forces_values[actuators[idx]]}")
    
await injectForce_axial_actuator(m2, actuators, f_Delta, sleep_time= 5)

In [ ]:
# get time for starting and ending telemetry data
time_start1 = datetime.now()
time_start1 = Time(time_start1, scale="utc")
print(f"UTC time to is {time_start} now.")

await asyncio.sleep(20)    

time_end1 = datetime.now()
time_end1 = Time(time_end1, scale="utc")


print(f"time start : {time_start1}")
print(f"time end : {time_end1}")

In [ ]:
# switch off actuators power (push E-stop)

#wait for pushing red button
command = input("Cut the power push E-stop")

In [ ]:
# retriveve telemetry with cut power

# get time for starting and ending telemetry data
time_start2 = datetime.now()
time_start2 = Time(time_start2, scale="utc")
print(f"UTC time to is {time_start2} now.")

await asyncio.sleep(120)    

time_end2 = datetime.now()
time_end2 = Time(time_end2, scale="utc")


#retrieve telemetry
names = list()
for idx in range(72):
    names.append(f"measured{idx}")    
    
data_act = await efd_client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=names,
        start=time_start1,
        end=time_end2)
data_act.to_csv("actuators_data.csv")

data_power_cell = await efd_client.select_time_series(
        "lsst.sal.MTM2.powerStatus",
        fields=["motorVoltage","motorCurrent","commVoltage","commCurrent"],
        start=time_start1,
        end=time_end2)
data_power_cell.to_csv("power_data.csv")


print(f"time start : {time_start1}")
print(f"time end : {time_end2}")

In [ ]:
#wait for restoring the power (release E-stop)
command = input("Restore the power, release E-stop")

await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#check the status
state = m2.evt_summaryState.get()
print(state)

In [ ]:
await m2.cmd_start.set_start(timeout = 30)

state = m2.evt_summaryState.get()
print(state)

In [ ]:
await m2.cmd_enable.set_start(timeout = 200)

state = m2.evt_summaryState.get()
print(state)